In [1]:
import gym 
import numpy as np
from gym.envs.toy_text.frozen_lake import FrozenLakeEnv
from typing import List

In [2]:
print(FrozenLakeEnv.__doc__)


    Winter is here. You and your friends were tossing around a frisbee at the park
    when you made a wild throw that left the frisbee out in the middle of the lake.
    The water is mostly frozen, but there are a few holes where the ice has melted.
    If you step into one of those holes, you'll fall into the freezing water.
    At this time, there's an international frisbee shortage, so it's absolutely imperative that
    you navigate across the lake and retrieve the disc.
    However, the ice is slippery, so you won't always move in the direction you intend.
    The surface is described using a grid like the following

        SFFF
        FHFH
        FFFH
        HFFG

    S : starting point, safe
    F : frozen surface, safe
    H : hole, fall to your doom
    G : goal, where the frisbee is located

    The episode ends when you reach the goal or fall in a hole.
    You receive a reward of 1 if you reach the goal, and zero otherwise.

    


In [3]:
env = gym.make('FrozenLake-v0')

### Helper Utilities 

In [4]:
# Typing
MDP = dict
State = int
Action = int 
Value = float
Policy = np.array
Optimal_Values = np.array

def show_state_space(markov_decision_process: MDP):
    return np.array(list(markov_decision_process.keys())).reshape(4, 4)

### Markov Decision Process (MDP)

In [5]:
env.env.P

{0: {0: [(0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 4, 0.0, False)],
  1: [(0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 4, 0.0, False),
   (0.3333333333333333, 1, 0.0, False)],
  2: [(0.3333333333333333, 4, 0.0, False),
   (0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False)],
  3: [(0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 0, 0.0, False)]},
 1: {0: [(0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 5, 0.0, True)],
  1: [(0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 5, 0.0, True),
   (0.3333333333333333, 2, 0.0, False)],
  2: [(0.3333333333333333, 5, 0.0, True),
   (0.3333333333333333, 2, 0.0, False),
   (0.3333333333333333, 1, 0.0, False)],
  3: [(0.3333333333333333, 2, 0.0, False),
   (0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False)]},
 2:

In [6]:
# Our State Space 
print(show_state_space(env.env.P))

[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]
 [12 13 14 15]]


In [7]:
# Our Action Space: "Left","Down","Right","Up"
env.env.P[3].keys()

dict_keys([0, 1, 2, 3])

In [8]:
state = 14
action = 3
transition_probability, next_state, reward, done = env.env.P[14][3][0]
print(f"Transition Probability to Goal State: {transition_probability} \nGoal State: {next_state}\nReward Function: {reward}\nEpisode Over: {done}")

Transition Probability to Goal State: 0.3333333333333333 
Goal State: 15
Reward Function: 1.0
Episode Over: True


### Value Iteration 

*Policy* $\pi(a | s) $: Specifies which action to take in a given state $s$. 

*Value Function:* Input state $s$ and outputs the value of being in that state $s$ under a particular policy $\pi$: $V^{\pi}: S \rightarrow \mathbb{R}$

<p>
$$V^{\pi}(s) = R^{\pi}(s) + \gamma \Sigma_{s' \in S} T(s' | s, \pi(a | s))V^{\pi}(s')$$
</p>

Note the recursive update. This can be represented as: 
<p>
$$
\begin{bmatrix}
           V^*(s_1) \\
           V^*(s_2)  \\
           \vdots \\
           V^*(s_n)
         \end{bmatrix} = \begin{bmatrix}
           R^*(s_1, a_1) \\
           R^*(s_2, a_2)  \\
           \vdots \\
           R^*(s_n, a_n)
         \end{bmatrix} + \gamma \begin{bmatrix}
    T(s_1, | s_1, \pi(s_1))       & T(s_2, | s_1, \pi(s_1)) & \dots & T(s_n, | s_1, \pi(s_1)) \\
    T(s_1, | s_2, \pi(s_2))       & T(s_2, | s_2, \pi(s_2)) & \dots & T(s_n, | s_2, \pi(s_1)) \\
    \vdots \ddots \\
    T(s_1, | s_n, \pi(s_n))       & T(s_2, | s_n, \pi(s_n)) & \dots & T(s_n, | s_n, \pi(s_n)) \\
\end{bmatrix}\begin{bmatrix}
           V^*(s_1) \\
           V^*(s_2)  \\
           \vdots \\
           V^*(s_n)
         \end{bmatrix}
$$
</p>

<p>
    $$ V^* = R^* + \gamma TV^* $$ 
</p>


Hence, we can derive an iterative algorithm for computing the value function: 

**Step 1**: Initialize $t=0$, $V^*_0(s)$. 

**Step 2**: 
- Repeat Until Convergence. 
    - For $s \in S$:
        - **Update the Value Function**: $$V^{*}_t(s_t) = \max_{a} \Sigma_{s_{t+1}} T(s_{t+1} | s_t, a_t)(R(s_t, a_t) + \gamma V^{*}_{t-1}(s_{t+1}))$$
        - **Update the Policy**: $$\pi^{*}_t(s_t) = \arg \max_{a}  \Sigma_{s_{t+1}} T(s_{t+1} | s_t, a_t)(R(s_t, a_t) + \gamma V^{*}_{t-1}(s_{t+1}))$$



In [9]:
def calculate_value_of_state_action(
    markov_decision_process: MDP,
    state: State,
    action: Action,
    state_values: np.array,
    discount: float
) -> Value:
    """
    Calculates expected value associated with being in a state and taking action
    """
    qvalue = 0
    for transition_probability, next_state, reward, done in markov_decision_process[state][action]:
        next_state_value = 0 if done else state_values[next_state] 
        qvalue += transition_probability * (reward + discount*next_state_value)
    return qvalue

In [10]:
assert calculate_value_of_state_action(env.env.P, 14, 3, np.zeros(16), 0.99) == 1/3

In [15]:
def calculate_value_of_state(
    markov_decision_process: MDP,
    state: State,
    state_values: np.array,
    discount: float,
) -> List[Value]:
    
    """
    Calculates a list of Q[s, a] values for given state
    """
    return [
        calculate_value_of_state_action(markov_decision_process, state, action, state_values, discount) 
        for action in markov_decision_process[state]
    ]


In [16]:
def value_iteration(
    markov_decision_process: MDP,
    discount: float = .99,
    threshold: float = 0.0000001,
) -> (Optimal_Values, Policy):
    """
    Calculates the Value Function for the Markov Decision Process
    """
    # Initialize the Value Function
    V = np.zeros(len(markov_decision_process.keys()))
    Pi = np.zeros(len(markov_decision_process.keys()))

    # Repeat Until Convergence
    while True:
        V_prev = V.copy()
        for state in markov_decision_process:
            V[state] = np.max(calculate_value_of_state(env.env.P, state, V_prev, discount))
            Pi[state] = np.argmax(calculate_value_of_state(env.env.P, state, V_prev, discount))

        if np.all(np.abs(V_prev - V) < threshold):
            break
    return V, Pi

In [17]:
correct_values = np.array(
    [0.542, 0.499, 0.471, 0.457, 0.558, 0., 0.358, 0., 0.592, 0.643, 0.615, 0., 0., 0.742, 0.863, 0.]
)

optimal_values, optimal_policy = value_iteration(env.env.P)
assert np.sum(np.abs(optimal_values - correct_values)) <= 0.01

### Policy Iteration 
- **Step 1**: Choose an arbitrary policy $\pi(a|s)$ 
- **Step 2**: Repeat until Convergence: 
    - **Policy Evaluation**: Compute the value function for $\pi$
        - **Approach 1**: Iterate the value function until convergence until $\pi$
        - **Approach 2**: Solve a linear program to compute:
        $$V^{\pi}_t(s_t) = \max_{a_t} \Sigma_{s_{t+1} \in S} T(s_{t+1} | s_t, a_t)(R(s_t, a_t) + \gamma V^{\pi}_{t-1}(s_{t+1}))$$
    - **Policy Improvement**: 
        $$\pi_{new}(s_t) = \arg \max_{a_t} \Sigma_{s_{t+1} \in S} T(s_{t+1} | s_t, a_t)(R(s_t, a_t) + \gamma V^{\pi}_{t-1}(s_{t+1}))$$

In [18]:
def policy_evaluation(
    markov_decision_process: MDP, 
    policy: Policy, 
    discount: float,
    threshold: float = 0.0000001
) -> List[Value]:
    
    """Calculates the Value Function for a Specific Policy 
    """
    
    # Initialize the Value Function 
    V = np.zeros(len(markov_decision_process.keys()))
    
    # Repeat Until Convergence 
    while True: 
        V_prev = V.copy()
        for state in markov_decision_process:
            V[state] = calculate_value_of_state_action(markov_decision_process, state, int(policy[state]), V_prev, discount)
        if np.all(np.abs(V_prev - V) < threshold):
            break
    return V

In [19]:
assert np.sum(np.abs(policy_evaluation(env.env.P, optimal_policy, 0.99) - optimal_values)) <= 0.001

**Exercise: Implement Policy Evaluation by Solving a Linear Program.**

In [25]:
def policy_improvement(
        markov_decision_process: MDP,
        policy: Policy,
        values: List[Value],
        discount: float,
) -> Policy:
    
    """Calculate the Policy Improvement Step 
    """
    for state in markov_decision_process:
        policy[state] = np.argmax(calculate_value_of_state(markov_decision_process, state, values, discount))
    return policy

In [23]:
def policy_iteration(markov_decision_process: MDP,
                     discount: float = 0.99,
                     threshold: float = 0.0001) -> Policy:
    
    """Implements Policy Iteration 
    """
    
    # Initialize a Random Policy
    num_actions = len(markov_decision_process[0])
    num_states = len(markov_decision_process.keys())
    policy = np.random.choice(np.arange(num_actions), num_states)

    # Repeat Until Convergence
    while True:
        policy_old = policy.copy()
        value_function_policy = policy_evaluation(env.env.P, policy, discount)
        policy = policy_improvement(env.env.P, policy, value_function_policy, discount)
        if np.sum(np.abs(policy_old - policy)) < threshold:
            break
    return policy

In [26]:
assert all(policy_iteration(env.env.P) == optimal_policy)